In [ ]:
import numpy as np
from src.petab.CRP2_CPE.scripts import create_CRP2_CPE_conditions, create_CRP2_CPE_Model, create_CPE_Model
from src.utils.params import ParameterContainer

%load_ext autoreload
%autoreload 2

In [ ]:
# Create the model
model_dir = "/PolyPESTO/src/petab/CRP2_CPE/"
model = create_CRP2_CPE_Model(model_dir=model_dir, force_compile=True)

In [ ]:
# Create the model
model_dir = "/PolyPESTO/src/petab/CRP2_CPE/"
model = create_CRP2_CPE_Model(model_dir=model_dir, force_compile=False)
# model = create_CPE_Model()

# Define a set of parameters to sweep (e.g. irreversible params, all params, base set of params, extended set, etc.)
pc = ParameterContainer.from_json("/PolyPESTO/src/data/parameters/CRP2_CPE.json")
pg = pc.get_parameter_group("IRREVERSIBLE")

# Define a set of conditions to generate synthetic data
t_eval = list(np.arange(0, 1, 0.1, dtype=float))
fA0s = np.array([0.25, 0.5, 0.75, 0.1], dtype=float)
cM0s = np.array([1.0, 1.0, 1.0, 1.0], dtype=float)
cond_df = create_CRP2_CPE_conditions(fA0s, cM0s)

# Generate and save the dataset
ds_name = "ds_0"
ds_dir = f"/PolyPESTO/src/data/datasets/CRP2_CPE/{ds_name}"

ds = model.generate_dataset(
    param_group=pg, t_eval=t_eval, cond_df=cond_df, name=ds_name
)#.write(ds_dir)
ds

In [ ]:
from src.utils.plot import plot_all_measurements
from matplotlib import pyplot as plt
import petab.v1.C as C

for id, df in ds.meas_dfs.items():
    plot_all_measurements(df, 
        group_by=C.SIMULATION_CONDITION_ID,
        # group_by=C.OBSERVABLE_ID,
        format_axes_kwargs={
            "set_xlabel": "Total Conversion", 
            "set_ylabel": "Monomer Conversion",
            "set_xlim": (0, 1),
            "set_ylim": (0, 1),
        },
        plot_style="both",
        alpha=0.5,
    )
    plt.show()